In [2]:
#23/8/24 Creates a K fold KNN model for each file in a folder provided they are chemopy

import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Chemopy/datasets/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        length = len(xtrain)
        nneigh = 100
        if length < nneigh:
            nneigh = length
        for mtry in range(1, nneigh):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = KNeighborsClassifier(n_neighbors=mtry, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = KNeighborsClassifier(n_neighbors=bestmetrics, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    finalresults.append(results[12])

For dataset NR-ER_Chemopy.csv
for fold 1 test set mcc of 0.5080553524011281 valid set mcc of 0.37707874867825847
for fold 2 test set mcc of 0.3357554274319713 valid set mcc of 0.3027688849089313
for fold 3 test set mcc of 0.3140747205739492 valid set mcc of 0.32672855120877364
for fold 4 test set mcc of 0.43164207000260835 valid set mcc of 0.25804189473943945
for fold 5 test set mcc of 0.3594425845307718 valid set mcc of 0.3936422464836486

validation metrics of:
positives in data 154
negatives in data 209
net accuracy = 0.7052341597796143
mcc = 0.39166046248198466
For dataset Developmental_Toxicity_Chemopy.csv
for fold 1 test set mcc of 0.3333333333333333 valid set mcc of 0
for fold 2 test set mcc of 0.16666666666666666 valid set mcc of 0
for fold 3 test set mcc of 0 valid set mcc of 0
for fold 4 test set mcc of 0 valid set mcc of 0.0
for fold 5 test set mcc of 0 valid set mcc of 0

validation metrics of:
positives in data 8
negatives in data 6
net accuracy = 0.42857142857142855
mcc =

for fold 5 test set mcc of 0.6080052052987127 valid set mcc of 0.6625413488689132

validation metrics of:
positives in data 45
negatives in data 49
net accuracy = 0.723404255319149
mcc = 0.45518969634282813
For dataset Ames_Mutagenicity_Chemopy.csv
for fold 1 test set mcc of 0.6273377055346614 valid set mcc of 0.5428103436529548
for fold 2 test set mcc of 0.5659344234092826 valid set mcc of 0.4922365450212216
for fold 3 test set mcc of 0.5530656357190423 valid set mcc of 0.6118159571255593
for fold 4 test set mcc of 0.6121976439855634 valid set mcc of 0.5216302253246087
for fold 5 test set mcc of 0.5784940083066806 valid set mcc of 0.5215995897786221

validation metrics of:
positives in data 840
negatives in data 657
net accuracy = 0.7969271877087508
mcc = 0.5856171905444313
For dataset Hepatotoxicity_Chemopy.csv
for fold 1 test set mcc of 0.40095930561862864 valid set mcc of 0.36570551902088977
for fold 2 test set mcc of 0.46634212223964394 valid set mcc of 0.28944959876782234
for fol

In [2]:
#23/8/24 Creates a K fold KNN model for each file in a folder provided they are chemopy

import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Chemopy/datasets/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

In [6]:
datasets[10]

,Toxicity_Value,Weight,nH,nHal,nHet,nHA,nF,nCl,nBr,nI,...,VSAEstate0,VSAEstate1,VSAEstate2,VSAEstate3,VSAEstate4,VSAEstate5,VSAEstate6,VSAEstate7,VSAEstate8,VSAEstate9
0,1,148.150,4,0,5,10,0,0,0,0,...,0.547454,13.559583,0.000000,1.539352,0.000000,0.000000,3.115231,0.000000,0.0,4.905046
1,0,392.198,16,6,16,28,6,0,0,0,...,79.365588,13.706120,6.623437,3.618548,-4.964313,0.501815,-4.874915,-0.226280,0.0,0.000000
2,0,249.040,7,2,9,16,0,2,0,0,...,0.000000,3.800942,8.238624,11.965103,0.176914,5.134145,0.000000,0.000000,0.0,11.906493
3,1,388.264,21,2,24,30,2,0,0,0,...,26.426309,14.554939,20.051051,2.034770,-1.100900,17.734904,-0.034407,0.000000,0.0,0.000000
6,1,342.253,27,0,20,27,0,0,0,0,...,7.947111,11.418944,15.242677,2.061951,1.967681,6.377731,10.473845,0.676726,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,0,394.289,22,0,22,31,0,0,0,0,...,2.010483,34.894734,10.864036,8.679544,-0.497035,12.393945,5.612816,1.541477,0.0,0.000000
1539,1,352.272,21,0,21,28,0,0,0,0,...,2.081292,22.837983,11.005589,3.737749,0.142924,13.350220,7.897791,2.113118,0.0,0.000000
1542,1,362.287,24,0,23,29,0,0,0,0,...,2.126948,18.888857,11.077501,3.931990,0.196565,19.636096,5.457594,2.351115,0.0,0.000000
1543,0,439.753,27,1,25,33,0,1,0,0,...,2.116340,26.333584,15.836239,3.421882,-0.736528,12.776628,4.340082,5.500209,0.0,6.356008


In [3]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Figure_Development/Chemopy/resultsdata/chemopy_knn.csv')

In [7]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Chemopy/datasets/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

In [11]:
datasets[14]

,Toxicity_Value,Weight,nH,nHal,nHet,nHA,nF,nCl,nBr,nI,...,VSAEstate0,VSAEstate1,VSAEstate2,VSAEstate3,VSAEstate4,VSAEstate5,VSAEstate6,VSAEstate7,VSAEstate8,VSAEstate9
0,0,168.060,14,0,6,12,0,0,0,0,...,0.000000,0.000000,52.176914,0.000000,0.000000,0.000000,-6.391944,-1.451636,0.0,0.000000
1,0,84.030,8,0,3,6,0,0,0,0,...,0.000000,0.000000,24.016204,0.000000,0.000000,0.000000,-0.953704,-0.729167,0.0,0.000000
2,1,222.045,5,0,3,15,0,0,0,0,...,0.000000,40.522482,25.547240,0.000000,0.000000,0.000000,-1.545556,-1.690833,0.0,0.000000
3,0,196.373,1,5,2,7,3,1,1,0,...,31.081597,0.000000,0.000000,0.000000,0.000000,0.000000,-4.317901,0.000000,0.0,6.680748
4,0,182.475,2,6,3,10,5,1,0,0,...,58.597099,0.000000,0.000000,-2.937500,0.000000,0.000000,-4.946443,-3.510069,0.0,4.324691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7480,1,158.139,9,0,12,13,0,0,0,0,...,0.000000,0.000000,6.375648,2.996481,0.000000,14.461204,0.000000,0.000000,0.0,0.000000
7481,1,198.182,9,0,10,14,0,0,0,0,...,15.932546,0.000000,1.142792,0.000000,0.896481,6.027971,2.172977,1.491392,0.0,1.502506
7482,1,264.242,14,0,22,22,0,0,0,0,...,0.000000,0.000000,10.542324,0.000000,0.000000,30.791009,0.000000,0.000000,0.0,0.000000
7483,1,256.219,12,0,20,21,0,0,0,0,...,5.926524,0.000000,8.160341,2.794728,0.000000,22.045884,0.572523,0.000000,0.0,0.000000


In [12]:
namelist[14]

'Ames_Mutagenicity_Chemopy.csv'